https://github.com/olferuk/Nautical

Preparation

In [ ]:
from nautical.bot import TelegramBot
from nautical.message import Message

bot = TelegramBot(token='<YOUR_TOKEN>', config_path='./config.db')

Create a bot like was shown in the snippet above. Your token can be acquired from the @bot_father in Telegram.  
Bot is running and ready to listen to your commands! Let's check:

Обращаемся к @botfather
- Нажимаем по Start или пишем ему команду /start
- /newbot
- Вводим имя. Может быть любым текстом с любыми символами (это не то имя, которое будет в урле, это отображаемое имя, когда открывается чат)
- Вводим никнейм. Это то, как найти бота, попадает в url, поэтому
только символы латинского алфавита и знак _

Получаем chat_id  
Это идентификатор чата, где будем общаться с ботом  

Заходим сюда:  
https://api.telegram.org/bot<здесь токен от BotFather>/getUpdates  
и подрезаем здесь chat id нашего приватного диалога.

Если видим такой результат: `{“ok”:true,"result":[]}`, то  
выполняем еще и следующие шаги:  
- Создаем группу с собой и женой, например.
- Добавляем бота. Важно, чтобы он не был добавлен сразу при
создании, а лишь добавлен уже после.
- Пишем что-нибудь.
- Переходим по той же ссылке, копируем оттуда chat id.
- Кикаем всех из беседы. Да, даже жену.

Библиотек масса, для отправки простых пушей нам подойдет Notifiers:  
pip install notifiers  
После чего напишем функцию для отправки сообщений send  

In [ ]:
!pip install notifiers

In [5]:
from notifiers import get_notifier

def make_telegram_notifier(
    token = ''
    , chat_id = ''):
    def wrapper(message):
        telegram = get_notifier('telegram')
        telegram.notify(message=message
                        , token=token
                        , chat_id=chat_id)
    return wrapper


In [6]:
send_telegram = make_telegram_notifier()
send_telegram('This is test message!')

notifiers шлет только текстовые сообщения  
Если нужно слать картинки

In [ ]:
!pip install telepot validators

In [16]:
import telepot
from validators import url as is_url

def make_telepot(
    token = ''
    , chat_id = ''):
    bot = telepot.Bot(token)
    def wrapper(message="", image=None):
        if image is None:
            bot.sendMessage(chat_id, message)
            return
        if not is_url(image):
            with open(image, 'rb') as image:
                bot.sendPhoto(chat_id, image, caption=message)
                return
        bot.sendPhoto(chat_id, image, caption=message)
    return wrapper

In [17]:
send_telegram = make_telepot()
send_telegram('test picture', 'test.png')

In [18]:
send_telegram = make_telepot()
send_telegram('test picture', 'https://avatars.mds.yandex.net/i?id=a8ffc42530d11d373e07ff512a9e4a966448e3dc-5179924-images-thumbs&n=13')

Пример нотификатора о работе какой-то функции 

In [41]:
import telepot
from validators import url as is_url
import time
from datetime import datetime


def telegram_timer(
    f # функция, которую мы декорируем
    , token = 'your_token'
    , chat_id = 'your_chart_id'):
    bot = telepot.Bot(token)
    def wrapper(*args, **kwargs):
        t_start = datetime.now()
        t_start_str = datetime.strftime(t_start, '%Y-%m-%d %H:%M:%S.%f')[:-3]
        res = f(*args, **kwargs)
        t_finish = datetime.now()
        t_finish_str = datetime.strftime(t_finish, '%Y-%m-%d %H:%M:%S.%f')[:-3]
        duration = str(t_finish - t_start)[:-3]
        message = f"Начал работать {t_start_str}\nЗакончил работать {t_finish_str}\nВремя выполения: {duration}"
        bot.sendMessage(chat_id, message)
        return res
    return wrapper

In [42]:
my_func = telegram_timer(time.sleep)
my_func(3)